# Building Data Lake for demographic analysis and prediction in the U.S.

## Data Engineering Capstone Project

### Project Summary
Main goal of the project is to building Data Lake for demographic analysis and prediction in the U.S.

There are several datasets with raw data:

* `I94 Immigration Data`;
* `U.S. City Demographic Data`;
* `Airport Codes`.

Project focuses on creating data model for the future analysis and building ETL-pipeline using Apache Spark. The whole ETL-pipeline processes raw data, cleanup, apply data quality checks and store to the analytical area in the Data Lake according to data model.
Resulting database allows analytics and data scientists to find and predict correlation between demographic situation in different cities/states and immigration to this locations.

The project follows the follow steps:

* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [3]:
# Do all imports and installs here
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope

We will build a database (in the analytical area of our data lake) for demographic analysis and predictions which depends on immegration data.
We already has `U.S. City Demographic Data` grouped by city for the 2015 year provided by [U.S. Census Bureau](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/information/). This will be the starting point.
We will extract U.S. immigration data grouped by year, month and city from `I94 Immigration Data` provided by [US National Tourism and Trade Office](https://travel.trade.gov/research/reports/i94/historical/2016.html).
Also we will use [Airport Code Table](https://datahub.io/core/airport-codes#data).

Our resulting database will have the following structure:

* `airports` – dimension table which contains data about U.S. international airports. This table filled from the raw dataset represented by [airport-codes_csv.csv](./airport-codes_csv.csv) data source file.
* `cities` – dimension table with U.S. cities that will interested us in the future analytics. This table filled from the raw dataset represented by [us-cities-demographics.csv](./us-cities-demographics.csv) datasource file. We want to analyze only data for the same cities to compare with aggregated data for the 2015 year.
* `calendar` – dimension table with year and month to have a possibility to monitor situation with month granularity. This table filled from the raw dataset represented by `I94 Immigration Data` SAS database.
* `immigration` – fact table which contains aggregated data grouped by year, month, and city. Each row of this table shows how much people (men, women and total) immgegrated to the particular U.S. city each month. This table filled from the raw dataset represented by `I94 Immigration Data` SAS database joined with `airports` and `cities` data.
* `demographics` – pre-aggregated materialized view which is good candidate to land in data mart and contains aggregated data grouped by year and city. It can be used for the fast analytics and reporting. This table filled in two steps:
  1. Filled from the `us-cities-demographics.csv` datasource file which already contains data for the 2015 year.
  2. Filled from the `immigration` fact table by additition to the data for the previous years which allows us to see growth of the population.

##### Important notes and assumptions

1. We assume that all immigrants from the `I94 Immigration Data` will come and stay in the U.S. Actually we could analyze their visa types to be more precises, but for the scope of this project we will not consider it.
2. We assume that all immigrants will stay for living in the city to which the airport belongs. In real life this may not be the case, but we will not consider it.
3. Actually `demographics` table will contains data only for the 2015 and 2016 years because the `I94 Immigration Data` dataset contains data only for the 2016 year but our pipeliens will work for datasets with following years too.

##### Tools

We will use [Apache Spark](https://spark.apache.org/) to build our pipeline. Also we will use Data Lake idea to build our resulting tables and store our data in the HDFS or object storage like S3. We will not create staging area because in the Data Lake approache we can just load raw datasets with Spark, make all necessary transformations, cleanup the data, apply data quality checks and land it in the analytical area.

#### Describe and Gather Data
Describe the data sets you're using. Where did it come from? What type of information is included?

In [5]:
# Read in the data here
file_name = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(file_name, 'sas7bdat', encoding="ISO-8859-1")

In [6]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness

Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.